In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool
import random
import gc
import os
import datetime
import pickle
import optuna
import sqlite3

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

myfavouritenumber = 0
seed = myfavouritenumber
random.seed(seed)

pd.set_option('max_rows', 9999)
pd.set_option('max_columns', 9999)

Half_Half_CatBoost Training

In [2]:
class Trainer:
    def __init__(self, importance_df, model_type='lgb', use_feature_num=None):
        self.model_type = model_type
        
        if use_feature_num is not None:
            self.features = importance_df['feature'][:use_feature_num].tolist()
        else:
            self.features = None
            
            
    def train_half_optuna(self, X_train, y_train, params, num_boost_round, early_stopping_rounds, verbose=200, trial=None):
        
#         if self.features is None:
#             self.features = X_train.columns
            
#         self.features = [c for c in self.features if c not in ['M']]
            
        self.X_train = X_train[self.features]
        self.y_train = y_train
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'rmse')
        
        if self.model_type == 'lgb':
            d_half_1 = lgb.Dataset(self.X_train[:int(self.X_train.shape[0] / 2)], 
                                   label=self.y_train[:int(X_train.shape[0] / 2)])
            d_half_2 = lgb.Dataset(self.X_train[int(self.X_train.shape[0] / 2):], 
                                   label=self.y_train[int(X_train.shape[0] / 2):])

            self.model_1 = lgb.train(params, train_set=d_half_1, num_boost_round=num_boost_round, 
                                     valid_sets=[d_half_2], verbose_eval=verbose, 
                                     early_stopping_rounds=early_stopping_rounds, 
                                     callbacks=[pruning_callback])
            
            oof = self.model_1.predict(self.X_train[int(self.X_train.shape[0] / 2):],
                                       num_iteration=self.model_1.best_iteration)
            oof = np.clip(oof, 0, a_max=None)
            
            rmse = np.sqrt(mean_squared_error(self.y_train[int(X_train.shape[0] / 2):], oof))
            
        elif self.model_type == 'cat':
            cat_features_index = np.where(self.X_train.dtypes == 'category')[0]
            d_half_1 = Pool(self.X_train[:int(self.X_train.shape[0] / 2)],
                            label=self.y_train[:int(self.X_train.shape[0] / 2)],
                            cat_features=cat_features_index)
            d_half_2 = Pool(self.X_train[int(self.X_train.shape[0] / 2):],
                            label=self.y_train[int(self.X_train.shape[0] / 2):],
                            cat_features=cat_features_index)

            params['iterations'] = num_boost_round
            self.model_1 = CatBoostRegressor(**params)
            self.model_1.fit(d_half_1, eval_set=d_half_2, use_best_model=True,
                        early_stopping_rounds=early_stopping_rounds, verbose=verbose)
            
            oof = self.model_1.predict(self.X_train[int(self.X_train.shape[0] / 2):])
            oof = np.clip(oof, 0, a_max=None)
            
            rmse = np.sqrt(mean_squared_error(self.y_train[int(X_train.shape[0] / 2):], oof))
            
        return rmse

In [3]:
# config
train_pkl_path = '../input/prep_train_20191126_list.pkl'
importance_path = '../Importance/importance_20191126.csv'
model_type = 'cat'

# Create SQLite Table
con = sqlite3.connect('ashrae_cat.db')

def objective(trial):
    # Search Parameter Range
    set_cat_params = {
        'learning_rate': 0.03,
        'eval_metric': 'RMSE',
        'loss_function': 'RMSE',
        'random_seed': 42,
        'task_type': 'GPU',
        'grow_policy': 'Lossguide',
        'depth': trial.suggest_int('depth', 4, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 50),
        'max_leaves': trial.suggest_int('num_leaves', 64, 300),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 1e-2, 1e+2),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
    }
    
    num_boost_round = 1000
    early_stopping_rounds = 50
    num_feature = 40
    verbose = 500
    
    # Train Model
    trainer = Trainer(importance_df=importance_df, model_type=model_type, use_feature_num=num_feature)
    rmse = trainer.train_half_optuna(train[0], train[1], set_cat_params, num_boost_round, 
                                       early_stopping_rounds, verbose, trial)

    return rmse

In [5]:
# Training
# Data Loading
with open(train_pkl_path, 'rb') as f:
    train = pickle.load(f)

importance_df = pd.read_csv(importance_path)

pruner = optuna.pruners.SuccessiveHalvingPruner(min_resource=50)

# New Study
# study = optuna.create_study(
#     study_name='ashrae_cat',
#     storage='sqlite:///ashrae_cat.db',
#     load_if_exists=True,
#     direction='minimize',
#     pruner=pruner
# )

# Reload Intermediate state
study = optuna.load_study(
    study_name='ashrae_cat',
    storage='sqlite:///ashrae_cat.db',
    pruner=pruner
)

study.optimize(objective, timeout=60*60*48)

0:	learn: 2.0182101	test: 2.0187884	best: 2.0187884 (0)	total: 717ms	remaining: 11m 55s
bestTest = 1.190000713
bestIteration = 163
Shrink model to first 164 iterations.


[I 2019-11-26 22:28:21,902] Finished trial#15 resulted in value: 1.1898062520794561. Current best value is 1.1898062520794561 with parameters: {'bagging_temperature': 0.11785699214569634, 'depth': 10, 'min_data_in_leaf': 23, 'num_leaves': 291, 'random_strength': 89}.


0:	learn: 2.0181711	test: 2.0187510	best: 2.0187510 (0)	total: 1.23s	remaining: 20m 29s
bestTest = 1.192581532
bestIteration = 172
Shrink model to first 173 iterations.


[I 2019-11-26 22:41:08,486] Finished trial#16 resulted in value: 1.1924530743135755. Current best value is 1.1898062520794561 with parameters: {'bagging_temperature': 0.11785699214569634, 'depth': 10, 'min_data_in_leaf': 23, 'num_leaves': 291, 'random_strength': 89}.


0:	learn: 2.0181272	test: 2.0188544	best: 2.0188544 (0)	total: 10.4s	remaining: 2h 53m 25s
bestTest = 1.189251722
bestIteration = 167
Shrink model to first 168 iterations.


...

[I 2019-11-28 21:50:37,058] Finished trial#100 resulted in value: 1.218108675794796. Current best value is 1.1712196391529086 with parameters: {'bagging_temperature': 0.03803178354860343, 'depth': 10, 'min_data_in_leaf': 23, 'num_leaves': 161, 'random_strength': 77}.


0:	learn: 2.0183512	test: 2.0189868	best: 2.0189868 (0)	total: 779ms	remaining: 12m 58s
bestTest = 1.200776542
bestIteration = 164
Shrink model to first 165 iterations.


[I 2019-11-28 22:07:02,894] Finished trial#101 resulted in value: 1.2006298415634464. Current best value is 1.1712196391529086 with parameters: {'bagging_temperature': 0.03803178354860343, 'depth': 10, 'min_data_in_leaf': 23, 'num_leaves': 161, 'random_strength': 77}.


0:	learn: 2.0189340	test: 2.0187854	best: 2.0187854 (0)	total: 1.07s	remaining: 17m 51s
bestTest = 1.194811477
bestIteration = 161
Shrink model to first 162 iterations.


[I 2019-11-28 22:22:54,329] Finished trial#102 resulted in value: 1.1945916025954089. Current best value is 1.1712196391529086 with parameters: {'bagging_temperature': 0.03803178354860343, 'depth': 10, 'min_data_in_leaf': 23, 'num_leaves': 161, 'random_strength': 77}.
